<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D1%83%D0%BF%D1%80%D0%B0%D0%B6%D0%BD%D0%B5%D0%BD%D0%B8%D1%8F/3.%20%D0%93%D1%80%D1%83%D0%BF%D0%BF%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/data/football.csv'

df = pd.read_csv(url)
df.head()

Предположим, что перед нами стоит задача посчитать сумму зарплат по клубам, чтобы найти клуб с самой высокой зарплатой.

Мы можем сделать это с помощью функции `groupby`:

In [ ]:
df.groupby(['Club'])

Мы передали на вход функции `groupby` колонку (или список колонок), по которой будет осуществляться группировка. Но в ответ получили что-то странное. Это объект группировки, он хранит в себе информацию о том, какие строки датафрейма (по индексным номерам) соответствуют определенной группе (в нашем случае определенному клубу). Увидеть это можно, вызвав у объекта группировки атрибут `groups`:

In [ ]:
df.groupby(['Club']).groups

Это означает, что `FC Köln` соответствуют следующие строки: `166, 379, 899, 1399, 3415, 6904, 7052, 8892, 12171, 12376`.

О чем нам это говорит? Нам — ни о чем, но для алгоритма группировки это важная информация. Он поймет, какие строки брать для включения в группу, когда мы решим применить к этим строкам какую-нибудь агрегирующую функцию, например функцию суммы `sum`:

In [ ]:
grouped_df = df.groupby(['Club']).sum()
grouped_df

Получился датафрейм, где все параметры, к которым можно было применить соответствующую агрегирующую функцию, были просуммированы. Если точнее, были просуммированы все численные параметры. Другие параметры, такие как национальность (`Nationality`), не будут отображены в датафрейме сгруппированных данных. Это связано с тем, что к таким колонкам нельзя применить агрегирующую функцию, потому что они содержат не численные параметры, а текст.

Давайте обратимся по индексу к какой-то из строк получившегося датафрейма:

In [ ]:
grouped_df.loc['Ajax']

Таким образом, мы получили серию, где индекс содержит все параметры, которые можно было просуммировать, а значения серии — суммы параметров для всех игроков `ФК "Ajax"`.

Теперь мы можем получить сумму зарплат игроков этого футбольного клуба:

In [ ]:
grouped_df.loc['Ajax']['Wage']

Тут важно заметить, что особого смысла суммировать другие параметры нет. Например, что нам даёт сумма возрастов? Это неинформативная метрика, а значит смысла её подсчитывать нет. Поэтому мы можем ограничить список наших параметров и при группировке осуществлять суммирование только по параметру `"Wage"` (заработная плата):

In [ ]:
grouped_df = df.groupby(['Club'])['Wage'].sum()
grouped_df

Уже лучше!

Но помните наш исходный вопрос: `"В каком клубе самая высокая зарплата?"`

Для ответа на этот вопрос нам достаточно лишь отсортировать полученные данные по убыванию суммы зарплат с помощью функции `sort_values`:

In [ ]:
# параметр ascending=False указывает, что данные надо сортировать по убыванию
grouped_df = df.groupby(['Club'])['Wage'].sum().sort_values(ascending=False)
grouped_df.head(5)

И это `Real Madrid`, что вполне ожидаемо!

Давайте рассмотрим ещё несколько примеров использования функции `groupby`.

Например, построим такую таблицу, где сгруппируем игроков по национальностям (`Nationality`) и посчитаем среднюю зарплату, средний возраст и среднюю силу удара. Ну а что, вдруг это всё как-то взаимосвязано?

Для подсчета среднего мы будем использовать агрегирующую функцию `mean`:

In [ ]:
df.groupby(['Nationality'])[['Wage','Age','ShotPower']].mean()

Далее отсортируем по убыванию усредненной зарплаты:

In [ ]:
df.groupby(['Nationality'])[['Wage','Age','ShotPower']].mean().sort_values(['Wage'], ascending=False).head(10)

Что?! Доминиканская республика? Очень странно, давайте поближе посмотрим на данные по этой стране:

In [ ]:
df.loc[df['Nationality'] == 'Dominican Republic'][['Name','Club','Wage','Age','ShotPower']]

Теперь всё ясно. В нашей выборке только два футболиста из этой страны, один из них играет в клубе `"Real Madrid"` и получает 140000 евро, а другой в испанском клубе второго дивизиона с зарплатой в 70 раз ниже! Поэтому среднее и получилось таким высоким.

### Задание 1

Посчитайте среднюю зарплату (`Wage`) и цену (`Value`) игроков разных позиций (`Position`). Представители какой позиции имеют самую высокую среднюю цену?

Какова средняя зарплата футболистов на данной позиции?

В предыдущих примерах мы использовали такие агрегирующие функции, как `sum` (сумма) и `mean` (среднее). Давайте рассмотрим, какие ещё функции можно использовать при группировке данных.

Функция `nunique`, которая позволяет посчитать количество уникальных значений по серии. Её лучше всего применять к тем колонкам датафрейма, в которых хранятся категорийные данные:

In [ ]:
df.groupby(['Nationality'])[['Club','Name']].nunique()

Функция `count`, с помощью которой можно посчитать количество элементов в группе. То есть результат её будет такой же, как у функции `value_counts`:

In [ ]:
df.groupby(['Club'])['Name'].count()

Функция `median`, с помощью которой можно посчитать медианное значение:

In [ ]:
df.groupby(['Club'])['Dribbling'].median()

Функция `max`, с помощью которой можно посчитать максимальное значение внутри группы:

In [ ]:
df.groupby(['Club'])['Strength'].max()

Функция `min`, с помощью которой можно посчитать минимальное значение внутри группы:

In [ ]:
df.groupby(['Club'])['Balance'].min()

### Задание 1

Посчитайте среднюю (`mean`) и медианную (`median`) зарплату (`Wage`) футболистов из разных клубов (`Club`). В скольких клубах средняя и медианная зарплаты совпадают?

Подсказка: чтобы в процессе группировки применить к данным одновременно две агрегирующие функции, необходимо указать их как аргументы метода `agg`:

```Python
df.groupby(столбец_для_группировки)[столбцы_для_отображения].agg(['функция_1', 'функция_2'])
```

### Задание 2

Продолжаем работать с клубами, в которых средняя зарплата совпадает с медианной. Каков максимальный размер средней зарплаты в этой группе клубов?

Как называется клуб, где игроки получают такую зарплату?

### Задача 1

С помощью функции groupby посчитайте сумму зарплат (`Wage`) футболистов клуба (`Club`) `"Chelsea"`.

### Задача 2

Определите максимальную зарплату футболиста национальности (`Nationality`) Аргентина (`"Argentina"`) в возрасте `20 лет`.

### Задача 3

Определите максимальную зарплату футболиста национальности (`Nationality`) Аргентина (`"Argentina"`) в возрасте `30 лет`.

### Задача 4

Определите минимальную зарплату футболиста национальности (`Nationality`) Аргентина (`"Argentina"`) в возрасте `30 лет`.

### Задача 5

Определите максимальную силу (`Strength`) и баланс (`Balance`) среди игроков клуба (`Club`) `"FC Barcelona"` из Аргентины (`"Argentina"`). Ответ через точку с запятой без пробела.